In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
df = pd.read_csv('NEW_VOTERS.csv')
df.head()

,Voter File VANID,LastName,FirstName,MiddleName,Suffix,PreferredEmail,Party,Age,AL_Support,CountyName,...,City,State,Zip5,Zip4,Cell Phone,CellPhoneIsCell,Preferred Phone,PreferredPhoneIsCell,Home Phone,HomePhoneIsCell
0,57812043,Peng,Jasmine,NaN,NaN,NaN,U,21,NaN,Santa Clara,...,San Jose,CA,95131.0,2755.0,NaN,NaN,4.082723e+09,Likely Not a Cell,4.082723e+09,Likely Not a Cell
1,59291016,Flores,Akira,NaN,NaN,NaN,D,19,NaN,Santa Clara,...,San Jose,CA,95131.0,2778.0,NaN,NaN,NaN,NaN,NaN,NaN
2,59700160,Brown,Thomas,NaN,NaN,NaN,U,19,NaN,Santa Clara,...,San Jose,CA,95131.0,2780.0,NaN,NaN,NaN,NaN,NaN,NaN
3,57983287,Xu,Xuewei,NaN,NaN,NaN,U,39,NaN,Santa Clara,...,San Jose,CA,95131.0,2784.0,NaN,NaN,NaN,NaN,NaN,NaN
4,59374587,Ansagay,Zhara,Michelle,NaN,NaN,D,18,NaN,Santa Clara,...,San Jose,CA,95131.0,2735.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.dropna(subset=['Zip5'], inplace=True)

In [56]:
df['FullAddress'] = df['Address'] + ' ' + df['City'] + ', ' + df['State'] + ' ' + df['Zip5'].astype(int).astype(str)
df['FullName'] = df['FirstName'] + ' ' + df['LastName']

In [72]:
# df2 = df.groupby('Full_Address').LastName.agg(['unique', 'nunique'])
df2 = df.groupby('FullAddress').agg({
    'LastName': ['unique', 'nunique'],
    'FullName': ['unique', 'nunique']
})

In [102]:
df3_multiple = df2.loc[df2['LastName']['nunique'] > 1].copy()
df3_multiple['AddressedTo'] = [', '.join(x[:-1] + ' and ' + x[-1]) + ' Families' for x in df3_multiple['LastName']['unique']]

In [103]:
df3_single_a = df2.loc[
    (df2['LastName']['nunique'] == 1)
     & (df2['FullName']['nunique'] == 1)
].copy()
df3_single_a['AddressedTo'] = [x[0] for x in df3_single_a['FullName']['unique']]

In [104]:
df3_single_b = df2.loc[
    (df2['LastName']['nunique'] == 1)
     & (df2['FullName']['nunique'] > 1)
].copy()
df3_single_b['AddressedTo'] = [', '.join(x[:-1]) + ' and ' + x[-1] for x in df3_single_b['FullName']['unique']]

In [105]:
df3 = pd.concat([
    df3_multiple['AddressedTo'],
    df3_single_a['AddressedTo'],
    df3_single_b['AddressedTo']
])

In [106]:
df3

FullAddress
100 Brook Ter Fremont, CA 94538                                    Lane and Laue Families
1000 Poda Ct Fremont, CA 94539                                  Gomez and Greeff Families
1001 E River Pkwy Santa Clara, CA 95054                          Vaccaro and Tow Families
1001 S Main St Apt F203 Milpitas, CA 95035                       Lewis and Singh Families
1001 S Main St Apt F312 Milpitas, CA 95035                      Khatun and Mukul Families
                                                                 ...                     
986 Sandalridge Ct Milpitas, CA 95035                       Ashok Sharma and Manju Sharma
988 Madeline Ln Santa Clara, CA 95050                      Judith Ehrat and Stephen Ehrat
990 Teal Dr Santa Clara, CA 95051                                   June Gan and Gary Gan
991 Teal Dr Santa Clara, CA 95051                           Jerry Scott and Dolores Scott
998 Garrity Way Santa Clara, CA 95054         Nicholas Desaulniers and Rachel Desaulnier

In [4]:
csvs = ['NEW_VOTERS', 'ONE_TIME_VOTERS', 'HIGH_PROPENSITY_DEMOCRATS']

for csv in csvs:
    print('='*75+'\n\n')

    # Import + Clean
    print('\n===== Importing + Cleaning ' + csv + ' =====\n')
    df = pd.read_csv(csv + '.csv')
    print(df['Address'].nunique(), 'unique addresses before dropna')
    df.dropna(how='any', subset=['Zip5', 'FirstName', 'LastName'], inplace=True)
    print(df['Address'].nunique(), 'unique addresses after dropna')

    # Create Columns
    # df['FullAddress'] = df['Address'] + ' ' + df['City'] + ', ' + df['State'] + ' ' + df['Zip5'].astype(int).astype(str)
    df['FullName'] = df['FirstName'] + ' ' + df['LastName']

    # Staging Aggregation
    print('\n===== Staging Aggregation =====\n')
    # df2 = df.groupby('FullAddress').agg({
    df2 = df.groupby(['Address', 'City', 'State', 'Zip5']).agg({
        'FirstName': ['unique'],
        'LastName': ['unique', 'nunique'],
        'FullName': ['unique', 'nunique']
    })
    print('Staging Aggregation successful.')

    # Final Aggregation
    print('\n===== Final Aggregation =====\n')

    # Find Houses w/ more than 9 tenants
    # print('-'*75+'\n')
    # anomalies = df2.loc[df2['FullName']['nunique'] > 9]
    # print(len(anomalies.index), 'addresses found with more than 9 tenants:')
    # for i, a in zip(anomalies.index, anomalies):
    #     print(i + '(' + str(anomalies['FullName']['nunique']) + '): ', anomalies['FullName']['unique'])
    # print('-'*75+'\n')

    # Multiple Families (3+ families) -- "Alex, Ariel, and Jason"
    df3_multiple = df2.loc[df2['LastName']['nunique'] > 2].copy()
    print(len(df3_multiple.index), 'addresses with 3+ tenants')
    df3_multiple['AddressedTo'] = [', '.join(x[:-1]) + ', and ' + x[-1] for x in df3_multiple['FirstName']['unique']]
    df3_multiple['NumPersons'] = df3_multiple['FullName']['nunique']
    print('-'*75)

    # Double Families (2 families, 3+ people) -- "Lee and Ongoco Families"
    df3_double_a = df2.loc[
        (df2['LastName']['nunique'] == 2) &
        (df2['FullName']['nunique'] > 2)
    ].copy()
    print(len(df3_double_a.index), 'addresses with double family tenants')
    df3_double_a['AddressedTo'] = [' and '.join(x) + ' Families' for x in df3_double_a['LastName']['unique']]
    df3_double_a['NumPersons'] = df3_double_a['FullName']['nunique']
    print('-'*75)

    # Double Persons (2 people) -- "Alex Lee and Ariel Ongoco"
    df3_double_b = df2.loc[
        (df2['LastName']['nunique'] == 2) &
        (df2['FullName']['nunique'] == 2)
    ].copy()
    print(len(df3_double_b.index), 'addresses with double individual tenants')
    df3_double_b['AddressedTo'] = [' and '.join(x) for x in df3_double_b['FullName']['unique']]
    df3_double_b['NumPersons'] = df3_double_b['FullName']['nunique']
    print('-'*75)
    
    # Single Families (1 family, 3+ persons) -- "Lee Family"
    df3_single_b = df2.loc[
        (df2['LastName']['nunique'] == 1)
        & (df2['FullName']['nunique'] > 2)
    ].copy()
    print(len(df3_single_b.index), 'addresses with single family (3+) tenants')
    df3_single_b['AddressedTo'] = [x[0] + ' Family' for x in df3_single_b['LastName']['unique']]
    df3_single_b['NumPersons'] = df3_single_b['FullName']['nunique']
    print('-'*75)

    # Single Families (1 family, 2 persons) -- "Alex and Darius Lee"
    df3_single_c = df2.loc[
        (df2['LastName']['nunique'] == 1)
        & (df2['FullName']['nunique'] == 2)
    ].copy()
    print(len(df3_single_c.index), 'addresses with single family (2) tenants')
    df3_single_c['AddressedTo'] = [' and '.join(x) + ' ' + y for x, y[0] in zip(df3_single_c['FirstName']['unique'], df3_single_c['LastName']['unique'])]
    df3_single_c['NumPersons'] = df3_single_c['FullName']['nunique']
    print('-'*75+'\n')

    # Single Persons (1 person) -- "Alex Lee"
    df3_single_a = df2.loc[df2['FullName']['nunique'] == 1].copy()
    print(len(df3_single_a.index), 'addresses with single individual tenants')
    df3_single_a['AddressedTo'] = [x[0] for x in df3_single_a['FullName']['unique']]
    df3_single_a['NumPersons'] = df3_single_a['FullName']['nunique']
    print('-'*75)

    # Concatenate all df3's
    df3 = pd.concat([
        df3_multiple[['AddressedTo', 'NumPersons']],
        df3_double_a[['AddressedTo', 'NumPersons']],
        df3_double_b[['AddressedTo', 'NumPersons']],
        df3_single_a[['AddressedTo', 'NumPersons']],
        df3_single_b[['AddressedTo', 'NumPersons']],
        df3_single_c[['AddressedTo', 'NumPersons']]
    ])
    print(len(df3.index), 'addresses total after final aggregation')
    print('-'*75+'\n')

    # Export to new CSV
    df3.to_csv(csv + '_DEDUPED.csv')
    print('Exporting ' + csv + ' successful.\n')

print('Done.')




===== Importing + Cleaning NEW_VOTERS =====

8596 unique addresses before dropna
8594 unique addresses after dropna

===== Staging Aggregation =====

Staging Aggregation successful.

===== Final Aggregation =====

9 addresses with 3+ tenants
---------------------------------------------------------------------------
8 addresses with double family tenants
---------------------------------------------------------------------------
304 addresses with double individual tenants
---------------------------------------------------------------------------
5 addresses with single family (3+) tenants
---------------------------------------------------------------------------
228 addresses with single family (2) tenants
---------------------------------------------------------------------------

8040 addresses with single individual tenants
---------------------------------------------------------------------------
8594 addresses total after final aggregation
----------------------------------